# Scraping articles from Il Fatto Quotidiano website

In [1]:
#I import the necessary packages to webscrape
import requests
from lxml.html import fromstring
import time
import pandas as pd

#I create a function to extract the information from the website
def fetch_article_html(url):
    response = requests.get(url)
    return response.text

#I create a function to extract the text of the article and to put together all of the paragraphs
def extract_article_text(html):
    tree = fromstring(html)
    article_text = tree.xpath("//section/p/text() | //section/p/strong/text() | //section[1]/div[1]/p/text() | //section[1]/div[1]/p/strong/text() | //section[1]/p/text() | //section[1]/p/strong/text()")
    return ' '.join(article_text)

#I define the url of the website and loop over the 27 pages present in the section I am interested in
MAXPAGES = 27
baseurl = "https://www.ilfattoquotidiano.it/category/elezioni-politiche-2022/page/"
allurls = [f"{baseurl}{i}/" for i in range(1, MAXPAGES + 1)]

#I create a list in which to store the information
articles_data = []

#I loop over every url and check if the page exist. Then I extract the link and title of each article from the website and 
#then for every link (so for every aerticle) I retrieve the test of that article. 
for url in allurls:
    try:
        #I check if the page exists
        response = requests.head(url)
        if response.status_code == 404:
            print("Page not found:", url)
            continue

        #I fetch the HTML source from the website
        response = requests.get(url)
        htmlsource = response.text

        tree = fromstring(htmlsource)

        #I find the links and titles
        article_links = tree.xpath("//div[1]/h3/a/@href | //div/h3/a/@href")
        article_title = tree.xpath("//div[1]/h3/a/text() | //div/h3/a/text()")

        #I extract the article texts
        for link, title in zip(article_links, article_title):
            try:
                #I fetch the HTML source of each article page
                article_html = fetch_article_html(link)

                #I extract the full article text
                article_text = extract_article_text(article_html)

                #I append the links, titles and text to the list created above
                articles_data.append({
                    'Link': link,
                    'Title': title,
                    'Text': article_text
                })

                #I delay of 1 second before the next request
                time.sleep(1)  

            except requests.exceptions.RequestException as e:
                print("Connection error:", e)
                continue

    except requests.exceptions.RequestException as e:
        print("Connection error:", e)
        continue

# I create a dataframe from the articles data dictionary
df = pd.DataFrame(articles_data)

# I print the DataFrame
print(df)

                                                  Link  \
0    https://www.ilfattoquotidiano.it/2022/10/03/la...   
1    https://www.ilfattoquotidiano.it/2022/10/01/el...   
2    https://www.ilfattoquotidiano.it/2022/09/30/ro...   
3    https://www.ilfattoquotidiano.it/2022/09/30/ef...   
4    https://www.ilfattoquotidiano.it/2022/09/29/ca...   
..                                                 ...   
265  https://www.ilfattoquotidiano.it/2022/09/08/el...   
266  https://www.ilfattoquotidiano.it/2022/09/08/dl...   
267  https://www.ilfattoquotidiano.it/2022/09/08/su...   
268  https://www.ilfattoquotidiano.it/2022/09/08/el...   
269  https://www.ilfattoquotidiano.it/2022/09/08/bo...   

                                                 Title  \
0    La previsione di Renzi sul Pd a La7: “Se Schle...   
1    Elezioni, Anpi: “Il nuovo governo onori il giu...   
2    Rosy Bindi a La7: “Pd si metta a disposizione ...   
3    Effetto flipper, come funziona il meccanismo d...   
4    Cacciari

In [4]:
#I transform the dataframe into a csv file, I add a new column and I save it 
articles_FattoQuotidiano = pd.DataFrame.from_dict(articles_data)
articles_FattoQuotidiano['News outlet'] = pd.Series('Il Fatto Quotidiano', index=articles_FattoQuotidiano.index)
articles_FattoQuotidiano.to_csv('articles_FattoQuotidiano.csv', index=False)
#I check that everything went smoothly
articles_FattoQuotidiano.head()

,Link,Title,Text,News outlet
0,https://www.ilfattoquotidiano.it/2022/10/03/la...,La previsione di Renzi sul Pd a La7: “Se Schle...,"“L’operazione fatta dal Pd , e in modo specif...",Il Fatto Quotidiano
1,https://www.ilfattoquotidiano.it/2022/10/01/el...,"Elezioni, Anpi: “Il nuovo governo onori il giu...","Il documento: ""Esito elettorale apre una fase ...",Il Fatto Quotidiano
2,https://www.ilfattoquotidiano.it/2022/09/30/ro...,Rosy Bindi a La7: “Pd si metta a disposizione ...,“Se il Pd non si mette a disposizione per su...,Il Fatto Quotidiano
3,https://www.ilfattoquotidiano.it/2022/09/30/ef...,"Effetto flipper, come funziona il meccanismo d...","Il sistema scatta quando i conti non tornano, ...",Il Fatto Quotidiano
4,https://www.ilfattoquotidiano.it/2022/09/29/ca...,Cacciari a La7: “Se Pd e M5s ora non trovano u...,“Tra Pd e 5 Stelle o c’è ora una qualche for...,Il Fatto Quotidiano


I create a condition with a list of rows and then use it to extract those rows from the dataset and put into another dataset, 
which will serve for the labeled dataset. Then, I drop those columns from the original dataset, as this is the unlabeled dataset. I save both datasets and reset their indexes.

In [5]:
condition = list(range(1, 20)) + list(range(132, 150)) + list(range(250, 268))
articlesFattoQuotidiano_labeled = articles_FattoQuotidiano.loc[condition].copy()
articlesFattoQuotidiano_labeled.head()
articlesFattoQuotidiano_labeled.to_csv('articlesFattoQuotidiano_labeled.csv', index=False)
articlesFattoQuotidiano_unlabeled = articles_FattoQuotidiano.drop(list(range(1, 20)) + list(range(132, 150)) + list(range(250, 270)))
articlesFattoQuotidiano_unlabeled.to_csv('articlesFattoQuotidiano_unlabeled.csv', index=False)
#I restore the indexes of both dataframes
articlesFattoQuotidiano_labeled.reset_index(drop=True, inplace=True)
articles_FattoQuotidiano.reset_index(drop=True, inplace=True)

In [6]:
#I check that everything went smoothly
articlesFattoQuotidiano_unlabeled

,Link,Title,Text,News outlet
0,https://www.ilfattoquotidiano.it/2022/10/03/la...,La previsione di Renzi sul Pd a La7: “Se Schle...,"“L’operazione fatta dal Pd , e in modo specif...",Il Fatto Quotidiano
20,https://www.ilfattoquotidiano.it/2022/09/27/el...,"Elezioni, Tajani dopo l’incontro con Meloni a ...",Si è tenuto nel pomeriggio di oggi un incontro...,Il Fatto Quotidiano
21,https://www.ilfattoquotidiano.it/2022/09/27/fr...,"Fratelli d’Italia, dopo la vittoria feste viet...","Secondo diversi retroscena giornalistici, a or...",Il Fatto Quotidiano
22,https://www.ilfattoquotidiano.it/2022/09/27/fr...,Fratoianni: “Renzi ha annunciato dialogo con M...,“ Renzi si è dichiarato disponibile a fare un...,Il Fatto Quotidiano
23,https://www.ilfattoquotidiano.it/2022/09/27/el...,"Elezioni, Majorino (Pd): “In periodo come ques...",“La destra che ha vinto è la peggiore d’Europa...,Il Fatto Quotidiano
...,...,...,...,...
245,https://www.ilfattoquotidiano.it/2022/09/10/re...,"Regionali Sicilia, chi è “Scateno” De Luca: la...",Gli ultimi sondaggi prima dello stop danno il ...,Il Fatto Quotidiano
246,https://www.ilfattoquotidiano.it/2022/09/10/el...,"Elezioni, Berlusconi scherza con Micciché: “L’...","“Stamattina guardando la mia gente, mi sono ac...",Il Fatto Quotidiano
247,https://www.ilfattoquotidiano.it/2022/09/10/ca...,Calenda contro il reddito di cittadinanza: “Pa...,"“Un infermiere che lavora 10 ore al giorno, ...",Il Fatto Quotidiano
248,https://www.ilfattoquotidiano.it/2022/09/10/ha...,“Hanno archiviato l’indagine su di me 3 anni f...,"Il caso di Alessandro Ferro, ex primo cittadin...",Il Fatto Quotidiano


In [7]:
#I check that everything went smoothly
articlesFattoQuotidiano_labeled

,Link,Title,Text,News outlet
0,https://www.ilfattoquotidiano.it/2022/10/01/el...,"Elezioni, Anpi: “Il nuovo governo onori il giu...","Il documento: ""Esito elettorale apre una fase ...",Il Fatto Quotidiano
1,https://www.ilfattoquotidiano.it/2022/09/30/ro...,Rosy Bindi a La7: “Pd si metta a disposizione ...,“Se il Pd non si mette a disposizione per su...,Il Fatto Quotidiano
2,https://www.ilfattoquotidiano.it/2022/09/30/ef...,"Effetto flipper, come funziona il meccanismo d...","Il sistema scatta quando i conti non tornano, ...",Il Fatto Quotidiano
3,https://www.ilfattoquotidiano.it/2022/09/29/ca...,Cacciari a La7: “Se Pd e M5s ora non trovano u...,“Tra Pd e 5 Stelle o c’è ora una qualche for...,Il Fatto Quotidiano
4,https://www.ilfattoquotidiano.it/2022/09/29/de...,De Masi a La7: “Pd dice di essere di sinistra ...,“Il M5s avrebbe preso più voti se ci fosse s...,Il Fatto Quotidiano
5,https://www.ilfattoquotidiano.it/2022/09/29/bi...,Biden allarmato dall’esito del voto in Italia:...,"Un messaggio, quello lanciato dal presidente a...",Il Fatto Quotidiano
6,https://www.ilfattoquotidiano.it/2022/09/28/ro...,Rosato (Italia viva): “Leader del partito unic...,“Con Meloni al governo faremo opposizione cost...,Il Fatto Quotidiano
7,https://www.ilfattoquotidiano.it/2022/09/28/pd...,"Pd, Verini: “Cambiare nome al partito è ultima...",“Come opposizione punteremo a dare delle risp...,Il Fatto Quotidiano
8,https://www.ilfattoquotidiano.it/2022/09/28/el...,"Elezioni, Zanni (Lega): “Risultato non soddisf...",“Il governo che si formerà dopo questa vittori...,Il Fatto Quotidiano
9,https://www.ilfattoquotidiano.it/2022/09/28/el...,"Elezioni, Soumahoro: “Faremo opposizione dentr...",“Grazie per averci e avermi dato fiducia. Port...,Il Fatto Quotidiano
